### Initial setups

In [ ]:
import gym
import neat
import numpy as np
import gym_sokoban
import os
import pickle
file_name = 'winner_test.pkl'

import neat
import visualize

# # 2-input XOR inputs and expected outputs.
xor_inputs = [(0.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 1.0)]
xor_outputs = [(0.0,), (1.0,), (1.0,), (0.0,)]


def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for xi, xo in zip(xor_inputs, xor_outputs):
            output = net.activate(xi)
            genome.fitness -= (output[0] - xo[0]) ** 2


def run():
    config_path = 'config-feedforward-3'
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    print(f"population: {p}")

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 300)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    for xi, xo in zip(xor_inputs, xor_outputs):
        output = winner_net.activate(xi)
        print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    node_names = {-1: 'A', -2: 'B', 0: 'A XOR B'}
    visualize.draw_net(config, winner, True, node_names=node_names)
    visualize.draw_net(config, winner, True, node_names=node_names, prune_unused=Fale)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

    # p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    # p.run(eval_genomes, 10)

run()

In [ ]:
import gym
import neat
import numpy as np
import gym_sokoban
import pickle
# import os
file_name = 'winner_test_2.pkl'

import neat
import visualize

outputs = [0,1,2,3,4,5,6,7,8]

def process_observation(environment):
    # Convert the observation to RGB frame or custom observation
    arr_walls, arr_goals, arr_boxes, arr_player = environment.render(mode='raw')

    # Initialize the combined array with walls (1s)
    combined = np.ones_like(arr_walls)
    
    # Set empty fields (0s)
    combined[arr_walls == 0] = 0
    
    # Set targets (3s)
    combined[arr_goals == 1] = 3
    
    # Set boxes (2s)
    combined[arr_boxes == 1] = 2
    
    # Set boxes on targets (4s)
    combined[(arr_boxes == 1) & (arr_goals == 1)] = 4
    
    # Set player position (5s)
    combined[arr_player == 1] = 5

    # Flatten the array
    flat_array = combined.flatten()
    
    return flat_array

# def ignore_same_state(

def fitness(genomes, config):
    env = gym.make('Sokoban-small-v1')
    for genome_id, genome in genomes:
        observation = env.reset()
        observation = process_observation(env)
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        fitness = 0
        done = False

        while not done:
            nn_actions = net.activate(observation)
            action = np.argmax(nn_actions)
            observation, reward, done, info = env.step(action)
            observation = process_observation(env)
            fitness += reward
        
        genome.fitness = fitness
    env.close()



# Load configuration
config_path = 'config-feedforward-4'
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                            neat.DefaultSpeciesSet, neat.DefaultStagnation,
                            config_path)

# Dynamically set the input size
env = gym.make('Sokoban-small-v1')
observation = env.reset()
# print(len(observation))
config.genome_config.num_inputs = len(observation)
# print(config.genome_config.num_inputs)

# Create the population
p = neat.Population(config)


# Add a reporter to show progress in the terminal
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(1))

# Run NEAT
winner = p.run(fitness, 50)

visualize.plot_stats(stats, ylog=False, view=True)
visualize.plot_species(stats, view=True)

# Save the winner
with open(file_name, 'wb') as f:
    pickle.dump(winner, f)


/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(



 ****** Running generation 0 ****** 



/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):
